Egenskaper ved nynorsk og bokmål

In [1]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
import pandas as pd

In [2]:
mu.css()

In [276]:
mu.code_toggle()

In [3]:
from dhlab.nbtokenizer import tokenize
from collections import Counter

In [4]:
def show_aggs(sums):
    d = sums.to_dict()
    return Markdown('\n\n'.join(["__" + x + "__:  `" + str(d[x]) + "`" for x in d]))
        

In [5]:
import sqlite3


ordbank = {
    "nob": "ordbank.db",
    "nno" : "ordbank_nno.db"
}


def get_paradigms(word, lang = 'nob'):
    res = query(ordbank[lang],"""
        select 
            substr(a.tag, 1, 4) as cat, 
            group_concat(distinct a.oppslag) 
        from 
            fullformsliste as a, 
            fullformsliste as b
            
        where 
            b.oppslag = ? 
            and b.lemma_id=a.lemma_id 
            
        group by
            cat""", (word,))
    result = []
    if res != []:
        for r in res:
            result.append([r[0].strip(), r[1].split(',')])
    return result

def get_paradigms_many(words, lang = 'nob'):
    with sqlite3.connect(ordbank[lang]) as con:
        cur = con.cursor()
        cur.execute("attach '' as words")
        cur.execute("create table words.wordlist (word varchar)")
        for word in words:
            cur.execute("insert into words.wordlist values (?)", (word,))
        res = cur.execute("""
            select 
                wl.word,
                substr(a.tag, 1, 4) as cat, 
                group_concat(distinct a.oppslag) 
            from 
                fullformsliste as a, 
                fullformsliste as b,
                words.wordlist as wl
            where 
                b.oppslag = wl.word
                and b.lemma_id=a.lemma_id 

            group by
                wl.word, cat""").fetchall()
        result = []
        if res != []:
            for r in res:
                result.append([r[0], r[1].strip(), r[2].split(',')])
    return result

def get_lemma(word, lang = 'nob'):
    res = query(ordbank[lang],"""
        select 
            distinct
            substr(a.tag, 1, 4) as cat, 
            l.GRUNNFORM
        from 
            fullformsliste as a, 
            lemma as l
        where 
            a.oppslag = ? 
            and l.lemma_id = a.lemma_id
        """, (word,))
    result = []
    if res != []:
        for r in res:
            result.append([r[0].strip(), r[1]])
    return result


def get_lemma_many(words, lang = 'nob'):
    with sqlite3.connect(ordbank[lang]) as con:
        cur = con.cursor()
        cur.execute("attach '' as words")
        cur.execute("create table words.wordlist (word varchar)")
        for word in words:
            cur.execute("insert into words.wordlist values (?)", (word,))
        res = cur.execute("""
        select 
            distinct
            wl.word,
            substr(a.tag, 1, 4) as cat, 
            l.GRUNNFORM
        from 
            fullformsliste as a, 
            lemma as l,
            words.wordlist as wl
        where 
            a.oppslag = wl.word
            and l.lemma_id = a.lemma_id
        """).fetchall()
    result = []
    if res != []:
        for r in res:
            result.append([r[0], r[1].strip(), r[2]])
    return result

def get_form(word, lang = 'nob'):
    res = query(ordbank[lang],"""
        select distinct
            tag
        from 
            fullformsliste 
        where 
            oppslag=?""", (word,))
    result = []
    if res != []:
        for r in res:
            result.append(r[0].strip())
    return result

def get_form_many(words, lang = 'nob'):
    with sqlite3.connect(ordbank[lang]) as con:
        cur = con.cursor()
        cur.execute("attach '' as words")
        cur.execute("create table words.wordlist (word varchar)")
        for word in words:
            cur.execute("insert into words.wordlist values (?)", (word,))
        res = cur.execute("""
            select distinct
                wl.word,
                f.tag
            from 
                fullformsliste as f,
                words.wordlist as wl
            where 
                f.oppslag=wl.word""")
        result = []
        if res != []:
            for r in res:
                result.append([r[0], r[1].strip()])
    return result

def get_variant(word, gramtype, lang = 'nob'):
    res = query(ordbank[lang],"""
        select distinct
            a.tag as cat, 
            a.oppslag 
        from 
            fullformsliste as a, 
            fullformsliste as b 
        where 
            b.oppslag = ? 
            and b.lemma_id=a.lemma_id
            and a.tag like ?
        group by
            cat""", (word, '%' + gramtype + '%'))
    result = []
    if res != []:
        for r in res:
            result.append((r[0].strip(), r[1]))
    return result

def query(db, sql, params = ()):
    with sqlite3.connect(db) as con:
        cur = con.cursor()
        res = cur.execute(sql, params).fetchall()
    return res


In [6]:
onob = ordbank['nob']
onno = ordbank['nno']

In [7]:
with sqlite3.connect(onob) as con:
    cur = con.cursor()
    cur.execute("attach '{onno}' as nno".format(onno=onno))
    cur.execute("select count(*) from fullformsliste as b, nno.fullformsliste as n where b.oppslag = n.oppslag ")
    res = cur.fetchall()
#res

In [8]:
mu.update("wordbank", silent=True)
import wordbank as wb

In [9]:
def check_numbers(ordbank, table, column):
    onob = ordbank['nob']
    onno = ordbank['nno']
    nob_ord = [x[0] for x in query(onob, "select distinct {column} from {table} where substr({column},1,1) != '-'".format(column = column, table = table))]
    nno_ord = [x[0] for x in query(onno, "select distinct {column} from {table} where substr({column},1,1) != '-'".format(column = column, table = table))]
    nno_serie = nb.frame(nno_ord).set_index(0)
    nno_serie['nno'] = 1
    nob_serie = nb.frame(nob_ord).set_index(0)
    nob_serie['nob'] = 1
    norsk = pd.concat([nob_serie, nno_serie], axis = 1).fillna(0)
    norsk['felles'] = norsk.nob*norsk.nno
    return norsk

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))

# Nynorsk og Bokmål ved Nasjonalbibliotekt



Nettbiblioteket 52 000 bøker med kode nno, og 460 000 med kode nob

Målfridprosjektet i samarbeid med Språkrådet.

Innhøsting av statlige domener på internet. Målt i antall løpende forekomster, er det 290 millioner nynorsk og 2,8 milliarder bokmål.

Norsk Ordbank - Hvor mange fullformer, hvor mange lemmaer?

In [86]:
fullform = check_numbers(ordbank, 'fullformsliste','oppslag')

In [76]:
lemma = check_numbers(ordbank, 'lemma', 'grunnform')

Dataene er i en tabell der cellene inneholder 1 om ordet finnes i respektive ordbok. Om ordformen er med i begge har felles-cellen også verdien 1.

__Fullformer__

In [380]:
fullform.sample(10)

,nob,nno,felles
merlespikar,0.0,1.0,0.0
dampkjøkkener,1.0,0.0,0.0
skihoppingane,0.0,1.0,0.0
tertiærtider,0.0,1.0,0.0
statiskest,1.0,0.0,0.0
fagfellevurderinga,0.0,1.0,0.0
fiskarbåt,0.0,1.0,0.0
koses,1.0,0.0,0.0
påliteleglaupa,0.0,1.0,0.0
hinderbaner,1.0,1.0,1.0


__Lemmaer__

In [382]:
lemma.sample(10)

,nob,nno,felles
anestesi,1.0,1.0,1.0
flæe,0.0,1.0,0.0
kommandoraid,1.0,1.0,1.0
heimepleier,1.0,0.0,0.0
etterhogst,1.0,1.0,1.0
Silver,0.0,1.0,0.0
sutte,1.0,1.0,1.0
jippo,1.0,1.0,1.0
in persona,1.0,1.0,1.0
a-våpen,1.0,1.0,1.0


In [78]:
from IPython.display import Markdown

In [286]:
shownum = lambda x, title='verdi', rounding = 2: Markdown(   title + str(round(x, rounding)))

Det er en betydelig forskjell i inventar mellom bokmål og nynorsk

Antall lemma, bokmål har under 75 % av antallet for nynorsk.

In [299]:
show_aggs(lemma.sum())

__nob__:  `74389.0`

__nno__:  `105042.0`

__felles__:  `47667.0`

Antall fullformer

In [300]:
show_aggs(fullform.sum())

__nob__:  `342977.0`

__nno__:  `400245.0`

__felles__:  `127387.0`

Ratioen fullform til lemma ligger noe over 4 for bokmål, og noe under 4 for nynorsk.

In [359]:
ordgruppe = ['beveg', 'bil', 'rød', 'lang' ]

In [360]:
paranob = get_paradigms_many(ordgruppe, lang = 'nob')

In [366]:
Markdown("Fullformer for _" + ', '.join(ordgruppe) + '_ :' + ', '.join({x for y in paranob for x in y[2]}))

Fullformer for _beveg, bil, rød, lang_ :lengre, lang, rødd, rødes, bevegete, biles, bilene, bevegede, bilende, rødeste, bevegende, langes, rødde, beveg, rødere, langer, beveges, biler, lengst, bevege, rødt, rødende, beveger, langa, rød, bevegd, lengste, bilt, langt, bilte, langete, langende, beveget, bil, lange, røde, bevega, bevegde, bilen, lenger, rødest, langet, langede, røder, bile

In [367]:
len({x for y in paranob for x in y[2]})

45

In [364]:
paranno = get_paradigms_many(ordgruppe, lang = 'nno')

In [368]:
Markdown("Fullformer for _" + ', '.join(ordgruppe) + '_ :' + ', '.join({x for y in paranno for x in y[2]}))

Fullformer for _beveg, bil, rød, lang_ :lengre, lang, rødd, langane, bevegast, beveg, bevegande, rødast, rødande, røda, langast, bevegar, biler, langar, bilar, bevege, lengst, langande, beveger, langa, rød, bevegd, lengste, bilt, langt, bilte, bila, bilande, bil, lange, røde, bevegt, bevega, bilane, bilast, langen, bevegde, bilen, rødt, rødde, røder, bile

In [369]:
len({x for y in paranno for x in y[2]})

42

Hvor mye er likt fra et ordboksperspektiv? Det er verdt å merke seg at det er betydelig overlapp, nynorsken har en større grad av egne ord enn bokmål

In [287]:
shownum(fullform.felles.sum()*100/fullform.nob.sum(), title = 'Bokmålsformer delt med nynorsk %: ', rounding=0)

Bokmålsformer delt med nynorsk %: 37.0

In [288]:
shownum(fullform.felles.sum()*100/fullform.nno.sum(), title = 'Nynorskformer delt med bokmål %: ', rounding = 0)

Nynorskformer delt med bokmål %: 32.0

In [289]:
shownum(lemma.felles.sum()*100/lemma.nob.sum(), "Felles lemma i % av bokmål ", rounding = 0)

Felles lemma i % av bokmål 64.0

In [290]:
shownum(lemma.felles.sum()*100/lemma.nno.sum(), "Felles lemma i % av nynorsk % ")

Felles lemma i % av nynorsk % 45.38

In [291]:
shownum(fullform.nob.sum()/lemma.nob.sum(), title = 'Fullformer pr. lemma i bokmål ', rounding = 1)

Fullformer pr. lemma i bokmål 4.6

In [292]:
shownum(fullform.nno.sum()/lemma.nno.sum(), title = 'Fullformer pr. lemma i nynorsk ', rounding = 1)

Fullformer pr. lemma i nynorsk 3.8

## Distribusjon i tekster

In [190]:
tekst = [x.lower() for x in tokenize("""Det skal ikkje meir til enn eit lite snøfall før byfolket er ute på glattisen
– Hjelp! 
Ei eldre dame med rullator står midt i svingen nedanfor der vi bur. Knea klaprar mot kvarandre, som ein nyfødd kalv står ho der, ho kjem ingen veg, verken att eller fram, beina skjelv, skjøret blafrar, og underlaget er såpe. Eg hjelper den gamle over gata og seier at ho ikkje må gå ut på slikt føre, men kva skal ein stakkar gjere når kaffiboksen er tom og kommunen ikkje akkurat renn ned dørene?

Langs gata stikk bakenden på bilane ut, det er uråd å få til ei anstendig parkering, sinte menn med raude ansikt rusar bilane og blå fingrar skrapar is. Nede på hovudvegen ligg det tre bilar i grøfta. Eg går midt i vegen med barnevogna, drosjesjåførane hyttar med neven og bussen tutar, det er kaos, det er Oslo, det er vinterlandet, johoo, vi som er fødde med ski på beina!

Det er det same kvar einaste gong. Meteorologane har vifta med pinnen, kika åtvarande på oss frå fjernsynet og sagt at det kjem. Vi lærer aldri. Kvar einaste gong står hovudstadsinnbyggjarane der og spinn og måkar, måkar og spinn, klagar og jamrar og hufsar og vassar, kva er dette, kva kan det vere, dette kvite teppet som kjem frå himmelen, hjelpeogtrøste, slik var det ikkje eingong i nittenfemtito!

Det er utruleg fascinerande å sjå kor overraska nordmenn kan verte av at nedbør faktisk kan falle som anna enn regn. Enkelte kunne ikkje ha vore meir overraska om dei hadde drege vekk gardinene og landet hadde vorte dekt av marsipan. Og nyheitssendingane vert fylt av kaos, tilgje oss, alle de nesten fire millionar nordmenn som ikkje bur i Oslo, men de berre må sjå på kor hjelpelause vi er, i sending etter sending, same maset år etter år. Det er verken vett i panna eller stål i bein og armar. Snøen burde halde seg i Skomakargata eller i det minste på Svalbard!

Eg må berre vedgå det; eg nyt det. Eg synest det er urkomisk. Det minner meg om den britiske ballettlærarinna mi som ikkje møtte opp til undervisning fordi det hadde kome sju centimeter snø. Ho rekna med at alt var avlyst. At bussane ikkje gjekk. At skuleungane sat heime. At butikken var stengd. Ho hadde endåtil opna naud-hermetikken ho hadde med seg frå Storbritannia, i tilfelle ho vart innesnødd. No har det jammen vorte slik blant nordmenn også! Her har kreative sjeler ei god muligheit for å tene store pengar. Naudrasjonar som kan lagrast i tilfelle snøfnugg. Eller kva med ”vi børstar av bilen din”-service? Her kunne kjekke handymenn reist rundt og måka, varma opp bilkupear og skrapa frontruter klokka sju om morgonen. For no har det vorte eit ork og eit slit og ein unntakstilstand.

– Kva er det du gliser slik for, bjeffa naboen, då eg kom litt svett, trillande med handleposar på slep. Han lurer på om det ikkje er tungt. – Jau, må eg medgå, det er tungt, men vi treng då trimmen, smiler eg, han synest eg er ein sadist. Byen er kvit og vakker og ungane hyler av fryd. Eg lagar ein engel i snøen i parken medan forbipasserande ser olmt på meg og lurer på om det er ironisk meint, eller om eg kanskje er ein slik kunstnar som gjer slikt berre for å sjå på reaksjonane.

Vintergleda finn eg i skiløypa. Ikkje av siklande nordmenn i kjekke trikotar med pulsklokke, men av eit par utlendingar på tydeleg lånte ski. Kjærastparet i altfor store grilldressar og polvottar har raude kinn og skrattar medan dei stavrar seg avgarde, som små born som tek sine fyrste, vaklande skritt. – Før beina, ikkje tramp, seier guten, jenta sklir baklengs og veivar med armane, ein trikot-tass kjem susande og sender eit irritert blikk mot desse sinkene som sperrar sporet, dei ensar han ikkje, dei er i si eiga verd, dei har ikkje eingong sett meg som smugkikar frå den blå swixen, det er vinterglede, den aller første turen med ski, det er magisk og det skjer i Oslo, i løypene, i vintrelandet.

Eg pakkkar inn jenta mi på åtte månader og set kursen mot lavande tre. I det ho ser pulken, set ho i eit kvin og viftar med armane, no er det tur, vi skal suse avgarde. Så får dei andre jamre seg alt dei vil. For vi må nyte det. Den vakraste vinteren i manns minne. Her. No. """)]

In [ ]:
tekst2 = [x.lower() for x in tokenize("""Han vaks opp på biblioteket og snakkar fire språk flytande. På vidaregåande vart han plassert i innvandrarklasse med lettlese pensum. Nå har han lagt fram Noregs fyrste heilskaplege språklov.

AV SIGRID SØRUMGÅRD BOTHEIM

Kulturminister Abid Raja nærast unnskyldar seg med det same han kjem inn i rommet.

– Nynorsk er eg ikkje så flink i!

Språkrådet er ikkje ute etter å teste nynorskkunnskapane til kulturministeren, men vi vil gjerne høyre meir om språklova, som han la fram i mai. Saman med lova kom det ei stortingsmelding som peikar ut område som er ekstra viktige for at språkpolitikken skal fungere. Opplæringssektoren, medie- og kultursektoren og IKT-politikken er nemnde som særlege satsingsområde i arbeidet med å tryggje norsk, samisk, nasjonale minoritetsspråk og norsk teiknspråk. Språkrådet vil få lovfesta oppgåver.

Ministerskifte og koronakrise
Det var ikkje sjølvsagt at språklova skulle bli lagt fram nettopp i 2020. Ei språklov har vore varsla heilt sidan stortingsmeldinga Mål og meining vart vedteken i 2008.

Men det vart Abid Raja som greidde å dra arbeidet i land. Den 12. mai 2020, på dagen 135 år etter jamstillingsvedtaket som sidestilte nynorsk og bokmål som offisielle språk i Noreg, la han fram språklova. På dette tidspunktet hadde Raja sete i ministerstolen i mindre enn fire månader. I mellomtida hadde også ein pandemi råka landet vårt.

– Under koronakrisen har det vore vanskeleg. Alt har vore stengt. Vi må koma med ulike kompensasjonspakker til kulturlivet. Heldigvis får vi også gjort ein del «normale» ting. Eg var nettopp i Snåsa for å leggje ned grunnsteinen til Saemien Sijte – Sørsamisk museum og kultursenter. Vi må jo halde fram med dei vanlege aktivitetane i departementet.

Har du fått sett ditt eige stempel på språklova?

– Da eg vart minister 24. januar, var det allereie gjort mykje bra arbeid med språklova. Det har Trine Skei Grande og departementet mykje av æra for. Det var lagt eit grunnlag, men det var ikkje teke politiske avgjerder ennå. Så eg måtte få saka gjennom i regjeringa, og nå står vi samla om lovforslaget.

Ei sterkare lovforankring vil heve statusen til språkpolitikken.
Abid Raja
Lova vart litt forseinka på grunn av koronasituasjonen, men ho vart lagd fram for Stortinget før sommarferien. Nå håpar kulturministeren at Stortinget vil handsama lova før jul.

– Eg er veldig oppteken av at vi skal få vedteke lova i år, slik at ho kan verke frå 1. januar 2021.

Må sikre at lova verkar
Vi veit at det er mange innbyggjarar i Noreg som ikkje får oppfylt dei språklege rettane sine i dag. Kulturministeren meiner at ei språklov både vil auke det allmenne språkmedvitet og gje Språkrådet meir autoritet.

– Ei sterkare lovforankring vil heve statusen til språkpolitikken. Gjennom språkpolitikken skal vi byggje eit einskapleg land. Eg er spent på korleis det går når lova skal drøftast i Stortinget, og eg er oppteken av at Språkrådet skal ha økonomiske musklar til å følgje opp språkpolitikken og språklova. Eg trur at språklova kan styrkje posisjonen til Språkrådet. Eg er glad for at lova slår fast at alle sektorar skal ta eit eige språkansvar, og at Kulturdepartementet skal ha eit koordineringsansvar.

Men korleis kan vi sørgje for at ulike sektorar tek ansvar for språket?

– Departementa har eit stort ansvar her. Mellom anna må departementa følgje opp arbeidet med norsk fagterminologi. Det er så lett å kopiere ord og omgrep som kjem frå utlandet, og som snik seg inn i språket på for eksempel oljeplattformer eller legekontor. Og plutseleg har språket i profesjonane fjerna seg frå språket i befolkninga. Vi må sørgje for at denne avstanden ikkje blir for stor. Vi har heldigvis mange arenaer som er viktige for å utvikle norsk fagspråk. Språkbanken til Nasjonalbiblioteket er viktig, biblioteka gjer sitt, skulane gjer sitt.

Språkleg mangfald er eit gode, ikkje eit problem.
Abid Raja
Raja legg vekt på at vi må bruke det norske språket i alle samanhengar.

– Vi skal ta vare på norsk. Samtidig meiner eg at språkleg mangfald er eit gode, ikkje eit problem. Eg meiner det er svært viktig å følgje opp språkmangfaldet i skulen. Det gjeld ikkje berre bokmål og nynorsk. Vi må også styrkje dei samiske språka, dei nasjonale minoritetsspråka og norsk teiknspråk. Noreg har dessutan fått ei minoritetsgruppe med barn som har andre språk som morsmål. Nå får elevane språkpoeng frå vidaregåande skule når dei søkjer om opptak til høgare utdanning. Det er ein stor verdi å lære seg framandspråk. Men vi må aldri gløyme kva som er fellesskapsspråka i Noreg. Det er dei språka denne lova vernar om.

Likestilling mellom bokmål og nynorsk
I formålsparagrafen til språklova står det at lova skal fremja likestilling mellom bokmål og nynorsk, og at det offentlege har eit særleg ansvar for å fremja nynorsk som det minst brukte norske skriftspråket. Men i dag er det eit godt stykke att til reell likestilling mellom målformene. Offentlege organ bryt den gjeldande mållova stadig vekk.

Korleis kan ei språklov endre situasjonen?

Raja ser bort på statssekretær Gunhild Berge Stang.

– Vi må passe på at Gunhild blir verande i Kulturdepartementet etter valet!

Raja fortel at han er kjend med at det skjer brot på mållova.

– Eg meiner at det går fram av språklova at nynorsk som mindretalsspråk krev ekstra styrking. Det er ikkje nok å tenkje likestilling, vi må tenkje særhandsaming for å kunna oppnå likestilling. Eg er glad for at fylkeskommunane også skal ha plikter. Det er ikkje lenger berre statsorgana som skal svara på nynorsk eller bokmål til folk som skriv til dei. Det skal også fylkeskommunane gjera.

Språkmektig statsråd
Lat oss avslutte der vi starta, med dei manglande nynorskkunnskapane til statsråden. Raja fortel at han gjekk i vanleg norskklasse på barneskulen og ungdomsskulen. Men da han begynte på vidaregåande, meinte skulen at han skulle gå i innvandrarklasse.

– Dette forstod eg ikkje før eg vart vaksen: På den tida fekk skulen meir betalt for elevar i innvandrarklasse med særundervisning. Skulane fekk tilskot på grunn av oss. Og nynorsk var sjølvsagt heilt utelukka i innvandrarklassene. Eg hadde ikkje noko i ei slik klasse å gjera, eg hadde gode karakterar i norsk. Eg vaks nærast opp på biblioteket.

’Når ein bannar på urdu, høyrest det ut som om ein kastar blomar!’ seier vi på panjabi.
Abid Raja
Men sjølv om sidemålsundervisninga har vore litt skranten, er kulturministeren meir språkmektig enn dei fleste. Han er fødd i Oslo og har pakistanske foreldre, og panjabi er morsmålet hans.

– Eg lærte fyrst panjabi, det er det mor mi snakkar. Far min ville også at eg skulle lære urdu, det er på ein måte «bokmålet» i Pakistan. Urdu er eit danna språk, medan panjabi er eit røft språk, mellom anna med svært fargerike bannord. «Når ein bannar på urdu, høyrest det ut som om ein kastar blomar!» seier vi på panjabi.

Norsk begynte Raja å lære i barnehagen da han var fire–fem år gammal. Dessutan snappa han opp ein del norske ord frå barne-tv. Etter kvart lærte han også hindi.

– Språk og dialekt rommar mykje kulturelt materiale og heng svært tett saman med identiteten vår. Om språket vårt skulle forsvinne, så vil ein del av landet vårt døy samtidig. Difor er det viktig å sikre norsk språk gjennom ei språklov.""")]

Ser på tekst fra Språkrådets nettsider: https://www.sprakradet.no/Vi-og-vart/Publikasjoner/Spraaknytt/spraknytt-42020/byggjer-samhald-med-spraklov/

In [236]:
tc = nb.frame_sort(nb.frame(Counter(tekst2), 'tekst'))

In [240]:
comp = pd.concat([fullform[['nob','nno']], tc.loc[[w for w in tc.index if w.isalnum()]]], axis = 1).fillna(0)

In [241]:
#comp['ukjent'] = 2 - comp[['nob', 'nno']].sum(axis = 1) 

In [242]:
nb.frame_sort(comp, 'tekst').head(50).astype(int).style.background_gradient(axis = 1)

,nob,nno,tekst
i,1,1,37
det,1,1,33
er,1,1,31
at,1,1,28
og,1,1,28
på,1,1,24
eg,0,1,20
vi,1,1,20
som,1,1,20
å,1,1,15


In [324]:
nb.frame_sort(comp[comp.tekst == 3], 'tekst',ascending = True).head(50).astype(int).style.background_gradient(axis = 1)

,nob,nno,tekst
ansvar,1,1,3
skulen,0,1,3
mykje,0,1,3
korleis,0,1,3
år,1,1,3
vårt,1,1,3
viktig,1,1,3
ut,1,1,3
svært,1,1,3
stortinget,1,1,3


In [243]:
import warnings
warnings.filterwarnings('ignore')

In [246]:
rent_nno = nb.frame_sort(comp[comp['tekst'] > 0][comp['nob'] == 0][comp['nno'] == 1], 'tekst').index

Hvor mye tekst er det totalt, merk at __nob__ og __nno__ bare står for type, ikke forekomst.

In [301]:
show_aggs(comp[comp.tekst > 0].sum())

__nob__:  `319.0`

__nno__:  `419.0`

__tekst__:  `1177.0`

Hvor mye er bare nynorsk? I teksten er 207 forekomster rene nynorskord så noe under 20 % består av ord som ikke finnes i bokmål.

In [302]:
show_aggs(comp.loc[rent_nno].sum())

__nob__:  `0.0`

__nno__:  `4.0`

__tekst__:  `5.0`

Hvor stor del av teksten utgjør topp 50? Omtrent 50 % av totalen - her har vi 586 forekomster av totalt 1177. For mindre tekster er utgjør topp 50 rundt 50 % av ordmengden, som kryper nedover etter hvert som tekstens størrelse øker.

In [303]:
show_aggs(nb.frame_sort(comp, 'tekst').head(50).astype(int).sum())

__nob__:  `41`

__nno__:  `47`

__tekst__:  `586`

## En bokmålstekst 

Også fra Språkrådet. https://www.sprakradet.no/Vi-og-vart/Publikasjoner/Spraaknytt/spraknytt-42020/sprakdagen-pa-tv/

In [256]:
tekst3 = [x.lower() for x in tokenize("""Fredrik Solvang har snakket med mange kloke folk om språkloven og språkets rolle i samfunnet. Hør hva de har på hjertet.

TEKST: ERLEND LØNNUM | FOTO: JOHNNY VAET NORDSKOG

Temaene
Språkdagen 2020 handlet om tre temaer: helsespråk, språk og digitalisering og mindretallsspråk i Norge.

Hvordan kan språkarbeid i helsesektoren komme pasientene til gode?
Hvordan kan språkteknologi forenkle hverdagen?
Hvordan kan det offentlige styrke det språklige mangfoldet i Norge?
Årets hovedtema var det felles ansvaret vi alle har for språket vårt. Temaet er aktuelt på grunn av den nye språkloven, som er til behandling i Stortinget. Den tilhørende stortingsmeldingen om språk understreker sektorenes eget ansvar for å gjennomføre norsk språkpolitikk. Språkrådet ønsket å sette søkelyset på de konsekvensene loven og stortingsmeldingen vil få for det offentlige Norge.

Programmet
NRK-journalist Fredrik Solvang ledet tre språkdebatter.
Solvang intervjuet kultur- og likestillingsminister Abid Raja. Se eget intervju med statsråden.
Språkdirektør Åse Wetås delte ut språkpris til assisterende helsedirektør Espen Rostrup Nakstad. Se egen sak.
Lege Wasim Zahid fortalte om legespråk versus pasientspråk.
Sametingspresident Aili Keskitalo fortalte om hvordan det er å være i språklig mindretall.
TV-sendingen – nå på YouTube
I år var Språkdagen primært et digitalt arrangement, som ble sendt direkte fra Det Norske Teatret i Oslo 3. november. Sendingen kan du se på YouTube. Den er tekstet og tolket til tegnspråk. Debattene er også lagt ut enkeltvis.


Språkdirektør Åse Wetås og NRK-journalist Fredrik Solvang var verter for Språkdagen 2020.

Språkdagen i bilder
DEBATT OM HELSESPRÅK

Solvang: I 2014 la helseminister Bent Høie fram «pasientens helsetjeneste», der det står at det skal satses på kommunikasjon, at pasienten skal settes i sentrum, og at digitale tjenester skal gjøre kontakten med helsevesenet enklere. Hva har skjedd siden disse målene ble presentert?

Forstår alle pasienter helseinformasjon om seg selv? Er alle brev fra sykehusene enkle å forstå? Hvor mye koster det egentlig samfunnet når pasienter misforstår og ikke kommer riktig forberedt til operasjon?


Fredrik Solvang i samtale med Vibeke Herikstad fra Helse sør-øst, Anne Kristin Vie fra Forbrukerrådet og Espen Rostrup Nakstad fra Helsedirektoratet.

Vibeke Herikstad | Foto: Johnny Vaet Nordskog

Helgelandssykehuset brukte to millioner kroner ekstra hvert år fordi mange pasienter ikke forstod innkallingsbrevene. Nasjonalt kan kostnaden være over 140 millioner kroner årlig.

Vibeke Herikstad 
Espen Nakstad | Foto: Johnny Vaet Nordskog

Vi bruker ikke ordet sykehus lenger i Norge. Vi har helseforetak. Men ganske få forstår forskjellen mellom et regionalt helseforetak og et helseforetak. Det siste er faktisk et sykehus.

Espen Rostrup Nakstad
DEBATT OM SPRÅK OG DIGITALISERING

Solvang: Språket er en viktig komponent i digitaliseringen, både det språket innbyggerne møter når de bruker digitale tjenester, og det språkarbeidet som ligger bak og gjør at systemene fungerer. Skal samtaleroboter være til hjelp, må de forstå spørsmålene, både på bokmål og nynorsk. Skjemaer på nett må være enkle å forstå, slik at vi klarer å fylle dem ut, og offentlige etater må lage felles løsninger som gjør at vi ikke trenger å fylle ut fem skjemaer med informasjon, men kanskje bare ett.


Fredrik Solvang i samtale med Cathrine Holten fra Digitaliseringsdirektoratet, Espen Sunde fra Nav, Lars Rønn fra Skatteetaten og Paul Chaffey fra Kommunal- og moderniseringsdepartementet.

Cathrine Holten | Foto: Johnny Vaet Nordskog

Vi må ofte sette jurister og tjenestedesignere sammen i et rom, og så må vi vente til det kommer hvit røyk ut. For da har de fått til løsninger som er forståelige for brukerne, samtidig som det er riktig nok.

Cathrine Holten
Lars Rønn | Foto: Johnny Vaet Nordskog

Vi hadde eksempler på at folk ikke forstod spørsmålet: Disponerer du boligen din? Brukertestene viste at flertallet svarte feil, at de ikke visste hva disponere betyr. Så vi måtte skrive om. Og det sier noe om hvilket nivå vi må tilrettelegge for.

Lars Rønn
Paul Chaffey | Foto: Johnny Vaet Nordskog

Digitalisering handler om å finne en balansegang mellom presisjonsnivå og forståelige begreper. Og da er det aller viktigste vi gjør, å bruke et klart og tydelig språk som folk forstår. Det må være et menneskevennlig språk som gjør at vi slipper å måtte ringe og spørre: Hva var det dere egentlig mente? For det koster masse ressurser i offentlig sektor.

Lars Rønn
DEBATT OM MINDRETALLSSPRÅKENE I NORGE

Solvang: Forslaget til språklov gir status og vern til språk i Norge. Dette er særlig viktig for de minste språkene som staten har ansvar for. I denne debatten har vi kalt dem mindretallsspråkene:

nynorsk som det minst brukte av de to norske skriftspråkene
de samiske språkene nordsamisk, sørsamisk og lulesamisk
norsk tegnspråk
minoritetsspråkene kvensk, romani og romanes
Flere av disse språkene er lite synlige i den norske offentligheten, men de som bruker dem, har rettigheter. Hvordan er egentlig stoda? Og hvordan jobber myndighetene for at alle språkbrukere skal få rettighetene sine oppfylt?

Jon Christian Fløysvik Nordrum | Foto: Johnny Vaet Nordskog

Med språkloven blir språkansvaret tydeligere fordi loven slår fast at norsk er det nasjonale hovedspråket i Norge. Det er nytt. Det er også presisert at nynorsk skal framheves særlig, for eksempel ved at også fylkeskommunene får en plikt til å kommunisere på begge målformer. Og så foreslås det en generell bestemmelse om at det offentlige skal kommunisere klart.

Jon Christian Fløysvik Nordrum
Petter Noddeland | Foto: Johnny Vaet Nordskog

Situasjonen for hørselshemma barn er verre nå enn for 30 år siden. Mange døveskoler er borte, og dermed finnes det færre språkarenaer for barna.

Petter Noddeland
Guri Melby | Foto: Johnny Vaet Nordskog

Det er fint at Språkdagen setter fokus på alle disse språkene i en sammenheng. Språkene har nemlig mange av de samme utfordringene og derfor også mange av de samme løsningene. Og når det gjelder det å skape språklige arenaer, har vi kommet lenger for samisk og nynorsk enn for norsk tegnspråk. Så der har vi en jobb å gjøre.

Guri Melby

Øystein Vangsnes fra Universitetet i Tromsø i samtale med kunnskapsminister Guri Melby.

INNLEGG OM SPRÅKLOVEN


Kultur- og likestillingsminister Abid Raja fortalte om språkloven.

Språkloven fremmer gode verdier for et liberalt demokrati. Den tar hensyn til ulike mindretall og erklærer staten som flerspråklig. Det gjør demokratiet vårt sterkere.

Abid Raja""")]

In [257]:
tc0 = nb.frame_sort(nb.frame(Counter(tekst3), 'tekst'))

In [260]:
comp0 = pd.concat([fullform[['nob','nno']], tc0.loc[[w for w in tc0.index if w.isalnum()]]], axis = 1).fillna(0)

In [241]:
#comp['ukjent'] = 2 - comp[['nob', 'nno']].sum(axis = 1) 

In [261]:
nb.frame_sort(comp0, 'tekst').head(50).astype(int).style.background_gradient(axis = 1)

,nob,nno,tekst
og,1,1,36
det,1,1,30
i,1,1,25
er,1,1,20
for,1,1,18
at,1,1,16
om,1,1,15
vi,1,1,15
å,1,1,13
til,1,1,12


In [264]:
rent_nno = nb.frame_sort(comp0[comp0['tekst'] > 0][comp0['nob'] == 0][comp0['nno'] == 1], 'tekst').index

In [268]:
rent_nob = nb.frame_sort(comp0[comp0['tekst'] > 0][comp0['nno'] == 0][comp0['nob'] == 1], 'tekst').index

Hvor mye tekst er det totalt, merk at __nob__ og __nno__ bare står for type, ikke forekomst.

In [304]:
show_aggs(comp0[comp0.tekst > 0].sum())

__nob__:  `364.0`

__nno__:  `255.0`

__tekst__:  `989.0`

Hvor mye er bare nynorsk? I teksten er 207 forekomster rene nynorskord så noe under 20 % består av ord som ikke finnes i bokmål.

In [305]:
show_aggs(comp0.loc[rent_nno].sum())

__nob__:  `0.0`

__nno__:  `4.0`

__tekst__:  `6.0`

De fire nynorskordene i teksten ser ut til å skyldes Norsk Ordbank, ikke teksten selv.

In [283]:
Markdown(', '.join(comp0.loc[rent_nno].index))

digitalisering, anne, helsesektoren, språkrådet

Men hvor mye er bare bokmål? Antall forekomster utgjør her ca 17 %, med 164 forekomster totalt av 989.

In [307]:
show_aggs(comp0.loc[rent_nob].sum())

__nob__:  `113.0`

__nno__:  `0.0`

__tekst__:  `164.0`

Hvor stor del av teksten utgjør topp 50? Omtrent 50 % av totalen, samme som for nynorskteksten - her har vi 461 forekomster av totalt 989. 

In [308]:
show_aggs(nb.frame_sort(comp0, 'tekst').head(50).astype(int).sum())

__nob__:  `39`

__nno__:  `33`

__tekst__:  `461`

Legg merke til at både nynorsk og bokmål er høyt oppe i begge tekstene. Nynorskteksten drar mot nynorsk, mens bokmålsteksten drar andre veien. Her tallene for topp 50 for nynorskteksten.

In [309]:
show_aggs(nb.frame_sort(comp, 'tekst').head(50).astype(int).sum())

__nob__:  `41`

__nno__:  `47`

__tekst__:  `586`

Hvordan bruke det her i videre arbeid med språknormer?

* Finne korresponderende ord
* Finne register
* Hvor mange hybridtekster er det?
* Kvantisere eksponering av språklig variasjon på et ordnivå
